In [1]:
%%bash
pip install seaborn --user


In [105]:
#General Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import sklearn

from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn import svm

#Autoenconder Functions
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.models import Model
from sklearn.model_selection import train_test_split

#Nueral Network Functions
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Data Preprocessing

In [3]:
#List of Missing Values
missing_values = ["?", "NaN",]
#Import Data
BCdata = pd.read_csv("dataR2.csv", delimiter=",",na_values = missing_values)
BCdata.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1


In [4]:
#Rename Data to fit algorithm
XData = BCdata
XData.dtypes

Age                 int64
BMI               float64
Glucose             int64
Insulin           float64
HOMA              float64
Leptin            float64
Adiponectin       float64
Resistin          float64
MCP.1             float64
Classification      int64
dtype: object

In [5]:
#Data Visualization
col = XData.select_dtypes("number").columns
col
XData[col]

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1
5,49,22.854458,92,3.226,0.732087,6.8317,13.679750,10.31760,530.410,1
6,89,22.700000,77,4.690,0.890787,6.9640,5.589865,12.93610,1256.083,1
7,76,23.800000,118,6.470,1.883201,4.3110,13.251320,5.10420,280.694,1
8,73,22.000000,97,3.350,0.801543,4.4700,10.358725,6.28445,136.855,1
9,75,23.000000,83,4.952,1.013839,17.1270,11.578990,7.09130,318.302,1


In [6]:
#Check for Missing Values
XData.isnull().sum()

Age               0
BMI               0
Glucose           0
Insulin           0
HOMA              0
Leptin            0
Adiponectin       0
Resistin          0
MCP.1             0
Classification    0
dtype: int64

In [7]:
#Data Info
XData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 10 columns):
Age               116 non-null int64
BMI               116 non-null float64
Glucose           116 non-null int64
Insulin           116 non-null float64
HOMA              116 non-null float64
Leptin            116 non-null float64
Adiponectin       116 non-null float64
Resistin          116 non-null float64
MCP.1             116 non-null float64
Classification    116 non-null int64
dtypes: float64(7), int64(3)
memory usage: 9.1 KB


In [8]:
#Separate into X and Y components for data processing
Y = XData['Classification']
X = XData.drop(columns=['Classification'])


# AUTOENCODER

In [169]:
#Split Data into Training and Testing (Change for experiments)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.05, random_state=0)

#Establish Encoder size (change for experiments)
esize = 5
#create input layer
input_layer = Input(shape=(X.shape[1],))
#create encoder layer
encoder_layer = Dense(esize, activation='relu')(input_layer)
 #create decoder layer 
decoder_layer = Dense(X.shape[1],activation = 'relu')(encoder_layer)
    
#create autoencoder and encoder models
Autoencoder = Model(input_layer, decoder_layer)
E = Model(input_layer, encoder_layer)
Autoencoder.compile(optimizer='adam', loss = 'mean_squared_error')
    
#Train Autoencoder   
Autoencoder.fit(X_train, X_train,
                epochs = 50,
                batch_size= 30,
                shuffle=True,
                verbose=1,
                validation_data=(X_test, X_test))

Train on 110 samples, validate on 6 samples
Epoch 1/50
110/110 [==============================] - 1s 6ms/step - loss: 33606.1719 - val_loss: 53641.2070
Epoch 2/50
110/110 [==============================] - 0s 121us/step - loss: 33221.2385 - val_loss: 52996.3320
Epoch 3/50
110/110 [==============================] - 0s 73us/step - loss: 32798.1934 - val_loss: 52334.3281
Epoch 4/50
110/110 [==============================] - 0s 65us/step - loss: 32402.0856 - val_loss: 51660.7461
Epoch 5/50
110/110 [==============================] - 0s 36us/step - loss: 31956.5206 - val_loss: 50987.5508
Epoch 6/50
110/110 [==============================] - 0s 73us/step - loss: 31521.4718 - val_loss: 50309.0508
Epoch 7/50
110/110 [==============================] - 0s 81us/step - loss: 31112.2559 - val_loss: 49619.2539
Epoch 8/50
110/110 [==============================] - 0s 72us/step - loss: 30660.4535 - val_loss: 48925.2344
Epoch 9/50
110/110 [==============================] - 0s 92us/step - loss: 30229.977

In [170]:
#Save the Learned features to use in the NN
learnedfeatures = E.predict(X_train)
ValidationFeatures = E.predict(X_test)
print(learnedfeatures)

[[1.83014259e+01 3.29837494e+02 0.00000000e+00 0.00000000e+00
  3.40897827e+02]
 [6.34042854e+01 2.78612518e+02 0.00000000e+00 0.00000000e+00
  3.06472198e+02]
 [7.90555954e+01 9.25517120e+01 0.00000000e+00 0.00000000e+00
  1.31371628e+02]
 [3.94953996e-01 3.09788391e+02 0.00000000e+00 0.00000000e+00
  3.32105225e+02]
 [4.06846046e+01 3.69968170e+02 0.00000000e+00 0.00000000e+00
  3.70875885e+02]
 [5.51138191e+01 2.40707626e+02 0.00000000e+00 0.00000000e+00
  2.64751038e+02]
 [1.00352524e+02 1.77988892e+02 0.00000000e+00 0.00000000e+00
  2.14307281e+02]
 [1.69801641e+00 3.79109406e+02 0.00000000e+00 0.00000000e+00
  3.83585297e+02]
 [7.91054459e+01 1.21545784e+02 0.00000000e+00 0.00000000e+00
  1.60718643e+02]
 [0.00000000e+00 4.47480347e+02 0.00000000e+00 0.00000000e+00
  4.64428314e+02]
 [0.00000000e+00 5.02317566e+02 0.00000000e+00 0.00000000e+00
  5.20740845e+02]
 [1.02251366e+02 9.14096680e+01 0.00000000e+00 0.00000000e+00
  1.30059311e+02]
 [7.38337173e+01 8.94068680e+01 0.000000

# Neural Network

In [171]:

NN = MLPClassifier(solver='sgd',alpha=1e-6, hidden_layer_sizes=(30,1),learning_rate ='adaptive', random_state=1)
NN.fit(learnedfeatures,Y_train)
NNPredicted = NN.predict(ValidationFeatures)
NNAccuracy = accuracy_score(Y_test,NNPredicted)*100
print(NNAccuracy)

50.0


c:\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# SVM

In [167]:
#SVM
SVM = svm.SVC(gamma='auto', kernel = 'linear', probability=True)
SVM.fit(learnedfeatures,Y_train)
SVMPredicted = SVM.predict(ValidationFeatures)
SVMAccuracy = accuracy_score(Y_test,SVMPredicted)*100
print(SVMAccuracy)

50.0
